In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
from patsy import dmatrices # 전처리를 위한 코드 -> 종속변수와 독립변수를 분리 
# 최소제곱법을 사용하기 위한 코드 LinearRegression 클래스는 예측값과 실제 값의 RSS를 최소화해 OLS(최소제곱법) 추정 방식으로 구현한 클래스
import statsmodels.api as sm; 
from statsmodels.stats.outliers_influence import variance_inflation_factor # VIF
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
%matplotlib inline

df = pd.read_csv('house_prices.csv')
df.head()

,house_id,neighborhood,area,bedrooms,bathrooms,style,price
0,1112,B,1188,3,2,ranch,598291
1,491,B,3512,5,3,victorian,1744259
2,5952,B,1134,3,2,ranch,571669
3,3525,A,1940,4,2,ranch,493675
4,5108,B,2208,6,4,victorian,1101539


# 독립 변수를 drop하지 않고 구한 OLS 회귀 결과

In [4]:
df['intercept'] = 1
lm = sm.OLS(df['price'], df[['intercept', 'bedrooms', 'bathrooms', 'area']])
results = lm.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.678
Model:                            OLS   Adj. R-squared:                  0.678
Method:                 Least Squares   F-statistic:                     4230.
Date:                Sat, 08 Jan 2022   Prob (F-statistic):               0.00
Time:                        01:43:04   Log-Likelihood:                -84517.
No. Observations:                6028   AIC:                         1.690e+05
Df Residuals:                    6024   BIC:                         1.691e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept   1.007e+04   1.04e+04      0.972      0.331   -1.02e+04    3.04e+04
bedrooms   -2925.8063   1.03e+04     -0.285      0.775    -2.3e+04    1.72e+04
bathrooms   7345.3917   1.43e+04      0.515      0.607   -2.06e+04    3.53e+04
area         345.9110      7.227     47.863      0.000     331.743     360.079
==============================================================================
Omnibus:                      367.658   Durbin-Watson:                   2.007
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              350.116
Skew:                           0.536   Prob(JB):                     9.40e-77
Kurtosis:                       2.503   Cond. No.                     1.16e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.16e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

bedrooms와 price의 coef(상관관계)가 -2925.8063임을 확인할 수 있다. bedrooms가 적어져야 price가 올라간다는 이야기인데 이것은 맞지 않다. 현실에서는 양의 관계를 보여주는 것이 데이터 분석에서는 음의 관계를 보여준다는 것은 다중공선성으로 인한 회귀 결과가 잘못 나온 것이다.

# VIF를 통해 다중공선성 파악

In [6]:
y, X = dmatrices('price ~ area + bedrooms + bathrooms', df, return_type = 'dataframe')

vif = pd.DataFrame()
# 이 코드는 행렬 X의 column(여기서는 area, bedrooms, bathrooms)를 순회하면서 해당 column(즉, 변수)의 VIF 값을 계산
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns 
vif

,VIF Factor,features
0,7.327102,Intercept
1,5.458190,area
2,20.854484,bedrooms
3,19.006851,bathrooms


일반적으로 VIF가 10이 넘으면 다중공선성이 있다고 판단하며 5가 넘으면 주의할 필요가 있음. 
bedrooms와 bathrooms의 VIF 계수가 10 이상이므로 서로 강한 상관 관계가 있음을 확인.
즉, 둘 중 하나를 제거(DROP)하고 회귀를 하면 다중공선성 문제 해결.

# Drop 이전 학습

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.3, random_state=0)

lr_afterdrop = LinearRegression()
lr_afterdrop.fit(X_train, y_train)
pred = lr_afterdrop.predict(X_test)
# R2 계산
print('R제곱 값:', r2_score(y_test, pred).round(2))


R제곱 값: 0.67


# Drop 이후 다중공선성 문제 해결 확인 

In [8]:
# 앞의 코드에서 bedrooms을 제거 
# 블로그에서는 bathrooms을 제거해주었지만 bedrooms을 제거해줘도 잘 됨.
lm = sm.OLS(df['price'], df[['intercept', 'bathrooms', 'area']])
results = lm.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.678
Model:                            OLS   Adj. R-squared:                  0.678
Method:                 Least Squares   F-statistic:                     6346.
Date:                Sat, 08 Jan 2022   Prob (F-statistic):               0.00
Time:                        01:51:42   Log-Likelihood:                -84517.
No. Observations:                6028   AIC:                         1.690e+05
Df Residuals:                    6025   BIC:                         1.691e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept   8215.9441   8063.295      1.019      0.308   -7590.999     2.4e+04
bathrooms   3834.7685   7223.513      0.531      0.596   -1.03e+04     1.8e+04
area         345.2352      6.827     50.566      0.000     331.851     358.619
==============================================================================
Omnibus:                      367.677   Durbin-Watson:                   2.008
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              350.604
Skew:                           0.536   Prob(JB):                     7.37e-77
Kurtosis:                       2.504   Cond. No.                     5.71e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.71e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# 다시 VIF를 통해 다중공선성 파악

In [9]:
y, X = dmatrices('price ~ area + bathrooms', df, return_type = 'dataframe')

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns 
vif

,VIF Factor,features
0,4.438137,Intercept
1,4.871816,area
2,4.871816,bathrooms


# Drop 이후 학습

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.3, random_state=0)

lr_afterdrop = LinearRegression()
lr_afterdrop.fit(X_train, y_train)
pred = lr_afterdrop.predict(X_test)
# R2 계산
print('R제곱 값:', r2_score(y_test, pred).round(2))


R제곱 값: 0.67


추가적으로, R-squared값은 bedrooms를 drop하지 않은 모델과 drop한 모델 모두 0.67입니다. R-squared 값은 결정 계수를 뜻하는 단어로 회귀모형 유용성의 척도입니다. R-squared가 drop 전과 후가 동일하게 0.67이라는 말은 bedrooms와 bathrooms 둘 모두가 필요하지 않다는 뜻입니다. 둘 중 하나만 있어도 된다는 겁니다. bedrooms를 drop했음에도 모델의 예측력에 영향을 끼치지 않았습니다.